In [1]:
import torch
import torchvision
from torchvision.transforms import v2
from torchvision.models import efficientnet_b0,EfficientNet_B0_Weights,densenet121,DenseNet121_Weights
from torch.utils.data import DataLoader
import skorch
from skorch.helper import predefined_split
from skorch.callbacks import Checkpoint,Freezer

c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feed

In [ ]:
#Data first preprocessed and then split into train/val/test with ratios 0.8/0.1/0.1
#Data preprocessing is augmentation and normalization. The data will be resized according to model input size then
#Rotation, shearing and zooming were used to increase dataset size from 372 to 4000 images. Then normalization was applied and
#Dataset was split using ratio 0.8/0.1/0.1 for train/valid/test set. Split-folder python library was used.
#First 10 layers of models were frozen, last 10 layers will be fine-tuned and final classifier layer changed to linear layer

#Models trained individually and then added to KONet, efficientnetbo dropout ratio of 0.1 and densenet121 dropout ratio of 0.3

#First 10 layers will be frozen and the rest will be fine tuned but paper says only last 10 layers will be fine tuned.
#Unknown what is considered as a layer and what isn't.
#We will instead freeze roughly the first half of the layers in the model.
#We will freeze first Conv2dNormActivation and first nine MBConv blocks

In [2]:
n_classes=2
image_shape=224
augmented_dataset_size=4000
path="D:\Osteoporosis detection\datasets\Osteoporosis Knee X-ray Dataset"
non_augment_transform=v2.Compose([v2.ToImageTensor(),
                       v2.ToDtype(torch.float32),
                       v2.Resize((image_shape,image_shape),antialias=True),
                       v2.Normalize(mean=[0],std=[1]),
                       ])
transforms=v2.Compose([v2.ToImageTensor(),
                       v2.ToDtype(torch.float32),
                       v2.RandomAffine(degrees=30,shear=30),
                       v2.RandomZoomOut(side_range=(1,1.5)),
                       v2.Resize((image_shape,image_shape),antialias=True),
                       v2.Normalize(mean=[0],std=[1]),
                       ])

In [3]:
non_augmented_dataset=torchvision.datasets.ImageFolder(path,transform=non_augment_transform)
dataset=torchvision.datasets.ImageFolder(path,transform=transforms)

In [4]:
factor=augmented_dataset_size//len(dataset)-1
remainder=augmented_dataset_size-(factor+1)*len(dataset)

In [5]:
new_dataset=torch.utils.data.ConcatDataset([non_augmented_dataset]+[dataset for _ in range(factor)])
del non_augmented_dataset,dataset

In [6]:
#dataset=torchvision.datasets.ImageFolder(path,transform=transforms)
generator1 = torch.Generator().manual_seed(42)
train_set,valid_set,test_set=torch.utils.data.random_split(new_dataset, [0.8,0.1,0.1], generator=generator1)

In [ ]:
image,label=train_set.__getitem__(0)

In [ ]:
image.shape

In [7]:
class KONet(torch.nn.Module):

    def __init__(
            self,
            m1_ratio=0.6,
            m2_ratio=0.4,
            m1_dropout=0.1,
            m2_dropout=0.3,
            n_classes=2
    ):
        super().__init__()
        assert m1_ratio+m2_ratio==1
        self.n_classes=n_classes
        self.m1_ratio=m1_ratio
        self.m2_ratio=m2_ratio
        self.m1_dropout=m1_dropout
        self.m2_dropout=m2_dropout

        self.efficient=efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        self.efficient.classifier[0]=torch.nn.Dropout(p=self.m1_dropout,inplace=True)
        self.efficient.classifier[-1]=torch.nn.Linear(in_features=1280,out_features=self.n_classes)

        self.dense=densenet121(weights=DenseNet121_Weights.DEFAULT)
        self.dense.classifier=torch.nn.Sequential(torch.nn.Dropout(p=self.m2_dropout,inplace=True),
                                            torch.nn.Linear(in_features=1024,out_features=n_classes),
                                            )

    def forward(self, x):
        m1=self.efficient(x)
        m2=self.dense(x)
        out=self.m1_ratio*m1+self.m2_ratio*m2
        return out

In [8]:
model_name='conv_next'

In [18]:
#EfficientNetB0 has 16 MBConv layers, freeze till 8th MBConv layer then. Freeze all till before 5th sequential
#DenseNet121 has 58 dense layers, freeze till 29th dense layer then. #Till before dense block 3
#Conv next tiny has 18 CNBlock
if model_name=='efficient':
    model=efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
    p=0.1
    model.classifier[0]=torch.nn.Dropout(p=p,inplace=True)
    model.classifier[-1]=torch.nn.Linear(in_features=1280,out_features=n_classes)
    frozen_layers=4
    freeze=['features.{}*.weight'.format(i) for i in range(frozen_layers)] + ['features.{}*.bias'.format(i) for i in range(frozen_layers)]
    
elif model_name=='dense':
    model=densenet121(weights=DenseNet121_Weights.DEFAULT)
    p=0.3
    model.classifier=torch.nn.Sequential(torch.nn.Dropout(p=p,inplace=True),
                                         torch.nn.Linear(in_features=1024,out_features=n_classes),
                                        )
    freeze=['features.conv0.weight','features.conv0.bias','features.norm0.weight','features.norm0.bias',
            'features.denseblock1.*.weight','features.denseblock1.*.bias','features.denseblock2.*.weight','features.denseblock2.*.bias',
            ]
    freeze+=['features.denseblock3.denselayer{}.*.weight'.format(i) for i in range(1,12)]
    freeze+=['features.denseblock3.denselayer{}.*.bias'.format(i) for i in range(1,12)]

elif model_name=='conv_next':
    p=0.3
    model=torchvision.models.convnext_tiny(weights='DEFAULT')
    model.classifier[2]=torch.nn.Sequential(torch.nn.Dropout(p=p,inplace=True),
                                        torch.nn.Linear(in_features=768,out_features=n_classes),
                                        )
    frozen_layers=5
    freeze=['features.{}*.weight'.format(i) for i in range(frozen_layers)]
    freeze+=['features.{}*.bias'.format(i) for i in range(frozen_layers)]

    freeze=['features.5.{}*.weight'.format(i) for i in range(2)]
    freeze+=['features.5.{}*.bias'.format(i) for i in range(2)]
elif model_name=='KONet':
    m1_ratio=0.6
    m2_ratio=0.4
    m1_dropout=0.1
    m2_dropout=0.3
    model=KONet(m1_ratio=m1_ratio,m2_ratio=m2_ratio,m1_dropout=m1_dropout,m2_dropout=m2_dropout,n_classes=n_classes)
    #Defines the blocks to be frozen
    m1_frozen_layers=4
    freeze=['efficient.features.{}*.weight'.format(i) for i in range(m1_frozen_layers)]
    freeze+=['efficient.features.{}*.bias'.format(i) for i in range(m1_frozen_layers)]

    freeze+=['dense.features.conv0.weight','dense.features.conv0.bias','dense.features.norm0.weight','dense.features.norm0.bias',
            'dense.features.denseblock1.*.weight','dense.features.denseblock1.*.bias','dense.features.denseblock2.*.weight',
            'dense.features.denseblock2.*.bias',
            ]
    freeze+=['dense.features.denseblock3.denselayer{}.*.weight'.format(i) for i in range(1,12)]
    freeze+=['dense.features.denseblock3.denselayer{}.*.bias'.format(i) for i in range(1,12)]

In [32]:
model.load_state_dict(torch.load(f'model/{model_name}best_param.pkl'))

<All keys matched successfully>

In [19]:
model

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=

In [20]:
monitor = lambda net: any(net.history[-1, ('valid_accuracy_best','valid_loss_best')])
cp=Checkpoint(monitor='valid_loss_best',dirname='model',f_params=f'{model_name}best_param.pkl',
               f_optimizer=f'{model_name}best_opt.pkl', f_history=f'{model_name}best_history.json')
cb = skorch.callbacks.Freezer(freeze)
classifier = skorch.NeuralNetClassifier(
        model,
        criterion=torch.nn.CrossEntropyLoss(),
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(valid_set),
        iterator_train=DataLoader,
        iterator_valid=DataLoader,
        iterator_train__shuffle=True,
        iterator_train__pin_memory=True,
        iterator_valid__pin_memory=True,
        #iterator_train__num_workers=1,
        #iterator_valid__num_workers=1,
        #iterator_train__persistent_workers=True,
        #iterator_valid__persistent_workers=True,
        batch_size=32,
        device='cuda',
        callbacks=[cp,cb],#Try to implement accuracy and f1 score callables here
        warm_start=True,
        )

In [21]:
classifier.fit(train_set,y=None,epochs=2)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ConvNeXt(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      )
      (1): Sequential(
        (0): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (1): Permute()
            (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
            (3): Linear(in_features=96, out_features=384, bias=True)
            (4): GELU(approximate='none')
            (5): Linear(in_features=384, out_features=96, bias=True)
            (6): Permute()
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): CNBlock(
          (block): Sequential(
            (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
            (

In [ ]:
test=torch.ones((3,3,image_shape,image_shape))
out=classifier.predict(test)
print("Output shape: ",out.shape)

In [34]:
from sklearn.metrics import roc_auc_score,f1_score
import numpy as np
classifier.initialize()
classifier.load_params(f_params=f'model/{model_name}best_param.pkl', f_optimizer=f'model/{model_name}best_opt.pkl', 
    f_history=f'model/{model_name}best_history.json')
print("Paramters Loaded")

Paramters Loaded


In [35]:
iterations=5
accuracy=[]
f1=[]
auc=[]
test_loader=DataLoader(valid_set,batch_size=8,shuffle=False)
for _ in range(iterations):
    probs=[]
    actual_labels=[]
    for test_features, actual_lb in iter(test_loader):
        prob=classifier.predict_proba(test_features)
        actual_lb=np.array(actual_lb)
        probs.append(prob)
        actual_labels.append(actual_lb)

    probs=np.concatenate(probs)
    pred_labels=np.argmax(probs,axis=1)
    actual_labels=np.concatenate(actual_labels)

    iteration_auc=roc_auc_score(actual_labels,probs[:,1])
    iteration_accuracy=np.mean(pred_labels==actual_labels)
    iteration_f1=f1_score(actual_labels,pred_labels)

    accuracy.append(iteration_accuracy)
    f1.append(iteration_f1)
    auc.append(iteration_auc)

print(model_name)

print(f"Accuracy mean: {np.mean(accuracy)} standard deviation: {np.std(accuracy)}")
print(f"F1-Score mean: {np.mean(f1)} standard deviation: {np.std(f1)}")
print(f"ROC_AUC  mean: {np.mean(auc)} standard deviation: {np.std(auc)}")

KONet
Accuracy mean: 0.9564516129032258 standard deviation: 0.0072928279388443686
F1-Score mean: 0.9567450931455129 standard deviation: 0.007315614513150931
ROC_AUC  mean: 0.9955118565644883 standard deviation: 0.0007550761790799909
